In [1]:
import pickle
import numpy as np


In [2]:
x_train = np.load('x_train.npy')
x_cv = np.load('x_cv.npy')

y_train = np.load('y_train.npy')
y_cv = np.load('y_cv.npy')

x_train.shape, x_cv.shape, y_train.shape, y_cv.shape

((74708, 64, 64, 3), (18677, 64, 64, 3), (74708, 1), (18677, 1))

In [3]:
with open('x_test_fake_mask.pickle', 'rb') as f:
    x_test_fake_mask = pickle.load(f)
    
with open('x_test_fake.pickle', 'rb') as f:
    x_test_fake = pickle.load(f)

In [4]:
len(x_test_fake_mask), len(x_test_fake)

(10, 10)

In [5]:
x_test_fake_mask[0].shape

(768, 1024)

In [6]:
import keras
from keras.models import load_model, Sequential
from keras.layers import Dense, Flatten
from keras.applications.resnet50 import ResNet50
import tensorflow as tf 

Using TensorFlow backend.


In [7]:

#tf.config.gpu.set_per_process_memory_growth(True)


vgg_model=keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(64,64,3))

model_aug=Sequential()
model_aug.add(vgg_model)

top_model=Sequential()
top_model.add(Flatten(input_shape=(2, 2, 512)))
#model_aug.add(Dropout(0.3))
top_model.add(Dense(64, activation='relu'))

top_model.add(Dense(1, activation='sigmoid'))

model_aug.add(top_model)

for layer in model_aug.layers[0].layers[:17]:
    layer.trainable=False

model_aug.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=1e-6), metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
import logging
logging.basicConfig(filename = 'vgg_16.log', filemode = 'w', format='%(name)s - %(levelname)s - %(message)s', level = logging.INFO)

class CustomLogging(keras.callbacks.Callback):

    def on_train_begin(self, logs={}):
        pass
 
    def on_epoch_begin(self, epoch, logs={}):
        logging.info("Epoch: {}".format(epoch))

In [9]:
#model_aug.load_weights(resume_weights)

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

callbacks = [EarlyStopping(monitor='val_loss', patience=2),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True),
             ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.000001, verbose=1),
             CustomLogging()]

model_aug = load_model('best_model.h5')

Instructions for updating:
Use tf.cast instead.


In [ ]:
history = model_aug.fit(x_train, y_train, epochs = 100, verbose = 1, validation_data=(x_cv, y_cv), callbacks=callbacks) #)

Train on 74708 samples, validate on 18677 samples
Epoch 1/100
74708/74708 [==============================] - 1389s 19ms/step - loss: 0.2676 - acc: 0.8898 - val_loss: 0.6396 - val_acc: 0.7600
Epoch 2/100
74708/74708 [==============================] - 1421s 19ms/step - loss: 0.2566 - acc: 0.8958 - val_loss: 0.6350 - val_acc: 0.7626
Epoch 3/100
74708/74708 [==============================] - 1354s 18ms/step - loss: 0.2462 - acc: 0.9013 - val_loss: 0.6293 - val_acc: 0.7649
Epoch 4/100
74708/74708 [==============================] - 1355s 18ms/step - loss: 0.2366 - acc: 0.9062 - val_loss: 0.6247 - val_acc: 0.7691
Epoch 5/100
74708/74708 [==============================] - 1358s 18ms/step - loss: 0.2274 - acc: 0.9111 - val_loss: 0.6202 - val_acc: 0.7725
Epoch 6/100
74708/74708 [==============================] - 1429s 19ms/step - loss: 0.2187 - acc: 0.9155 - val_loss: 0.6165 - val_acc: 0.7734
Epoch 7/100
74708/74708 [==============================] - 1347s 18ms/step - loss: 0.2106 - acc: 0.9197 

In [ ]:
logging.basicConfig(filename = 'vgg_16.log', filemode = 'w', format='%(name)s - %(levelname)s - %(message)s', level = logging.INFO)

In [ ]:
model_aug.save("vgg16.h5")
logging.info("Saved model to disk")

In [ ]:
with open('vgg16_history.pickle', 'wb') as f:
    pickle.dump(history, f)

In [ ]:
logging.info('dumped history into pickle')